In [17]:
# Imports

import os

from zipfile import ZipFile

import arcpy

In [18]:
# Vars

base_dir = r"C:\Users\Misti\OneDrive\PROJECTS\2025_08_Multi-Year_Census_Counties"

output_gdb = r"2025_08_Multi-Year_Census_Counties.gdb"

output_fc = "multi_year_census_counties"

zip_dir_name = r"Zipped"

unzip_dir_name = r"Unzipped"

found_fcs = []

year_fcs_dict = {}

In [19]:
# Derived vars

full_output_gdb = os.path.join(base_dir, output_gdb)

full_output_fc = os.path.join(full_output_gdb, output_fc)


In [20]:
def extract_all_zips():

    for file in os.listdir(os.path.join(base_dir, zip_dir_name)):

        if not file.endswith(".zip"):
            continue
                        
        with ZipFile(os.path.join(base_dir, zip_dir_name, file), 'r') as zip_obj:
            zip_obj.extractall(os.path.join(base_dir, unzip_dir_name))

# extract_all_zips()

In [21]:
def get_counties(found_fcs):

    unfound = 0
    unfound_gdbs = []

    # Reset list in case of multiple script/block runs (duplicate paths not needed LOL)
    found_fcs.clear()

    for file in os.listdir(os.path.join(base_dir, unzip_dir_name)):

        if not file.endswith(".gdb"):
            continue

        gdb = os.path.join(base_dir, unzip_dir_name, file)
        arcpy.env.workspace = gdb
        
        if not "County" in arcpy.ListFeatureClasses():
            unfound +=1
            unfound_gdbs.append(gdb)
        else:
            found_fcs.append(os.path.join(base_dir, unzip_dir_name, file, "County"))

    if unfound:
        print(f"Heads up! Didn't find a 'County' FC in these GDBs: {unfound_gdbs}")
    else:
        print("Yay! Found a 'County' FC in all gdbs!")

    return found_fcs

get_counties(found_fcs)


Yay! Found a 'County' FC in all gdbs!


['C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2013_a_us_substategeo.gdb\\County',
 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2014_a_us_substategeo.gdb\\County',
 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2015_a_us_substategeo.gdb\\County',
 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2016_a_us_substategeo.gdb\\County',
 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2017_a_us_substategeo.gdb\\County',
 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2018_a_us_substategeo.gdb\\County',
 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2019_a_us_substategeo.gdb\\County',
 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2020_a_us_su

In [22]:
def create_dict(year_fcs_dict):

    # Resetting intermediate and final lists/dicts
    # While testing/developing
    pre_dict = {}
    pre_list = []

    print(year_fcs_dict)

    year_fcs_dict.clear()

    for fc in found_fcs:

        year = fc.split("tlgdb_")[1].split("_a_")[0]
        pre_dict[year] = fc

    pre_list = sorted(pre_dict.items(), reverse=True)
    year_fcs_dict = dict(pre_list)

    return year_fcs_dict

create_dict(year_fcs_dict)

{}


{'2025': 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2025_a_us_substategeo.gdb\\County',
 '2024': 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2024_a_us_substategeo.gdb\\County',
 '2023': 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2023_a_us_substategeo.gdb\\County',
 '2022': 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2022_a_us_substategeo.gdb\\County',
 '2021': 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2021_a_us_substategeo.gdb\\County',
 '2020': 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2020_a_us_substategeo.gdb\\County',
 '2019': 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\Unzipped\\tlgdb_2019_a_us_substategeo.gdb\\County',
 '2018': 'C:\\Users\\Misti\\OneDrive\\PROJECTS\\

In [23]:
def create_output():

    spatial_ref = arcpy.Describe(found_fcs[0]).spatialReference

    if not arcpy.Exists(full_output_gdb):
        print("Creating output GDB...")
        arcpy.management.CreateFileGDB(base_dir, output_gdb)
    else:
        print("Yay! Output GDB already exists.")

    if not arcpy.Exists(full_output_fc):
        print("Creating output feature class...")
        arcpy.management.CreateFeatureclass(full_output_gdb, output_fc, "POLYGON", template=found_fcs[0], spatial_reference=spatial_ref)
    else:
        print("Yay! Output feature class already exists.")

create_output()

Yay! Output GDB already exists.
Yay! Output feature class already exists.


In [24]:
def add_year_field():
    
    # I wouldn't technically have to do this bit because
    # If the field already exists, arcpy will just do nothing and move on...
    if not "Census_Year" in (f.name for f in arcpy.ListFields(full_output_fc)):
        
        print("Adding 'Census_Year' field")
        arcpy.management.AddField(full_output_fc, "Census_Year", "SHORT", field_length=4, field_alias="Census Year")

    else:
        print("Field 'Census_Year' alread exists! Moving on...")

add_year_field()

Field 'Census_Year' alread exists! Moving on...


In [ ]:
def append_counties(found_fcs):
    
    arcpy.management.Append(found_fcs, full_output_fc, schema_type='NO_TEST')

append_counties(found_fcs)

In [ ]:
# EVERYTHING BELOW THIS BLOCK IS NOT PART OF THE ACTUAL SCRIPT!! TESTING / DEV BLOCKS ONLY!!

In [25]:
# FOR USE WHILE BUILDING SCRIPT AND TROUBLESHOOTING ONLY!!!

def truncate_table():
    
    arcpy.management.TruncateTable(full_output_fc)

truncate_table()